In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd

train = pd.read_csv('../data/preliminary/train.csv')
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             240000 non-null  int64  
 1   trans_date_trans_time  240000 non-null  object 
 2   cc_num                 240000 non-null  int64  
 3   merchant               240000 non-null  object 
 4   category               240000 non-null  object 
 5   amt                    240000 non-null  float64
 6   first                  240000 non-null  object 
 7   last                   240000 non-null  object 
 8   gender                 240000 non-null  object 
 9   street                 240000 non-null  object 
 10  city                   240000 non-null  object 
 11  state                  240000 non-null  object 
 12  zip                    240000 non-null  int64  
 13  lat                    240000 non-null  float64
 14  long                   240000 non-nu

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,1072349,2020-03-21 19:05:27,2242542703101233,fraud_Bahringer Group,health_fitness,1.40,Samuel,Jenkins,M,43235 Mckenzie Views Apt. 837,...,38.4921,-85.4524,564,Pensions consultant,1996-04-10,5a4c5bc47fd9ca60a9cdbe38423654b4,1363892727,37.965322,-85.661501,0
1,1123019,2020-04-12 21:34:21,4469777115158234880,"fraud_Douglas, Schneider and Turner",shopping_pos,9.73,Gregory,Graham,M,4005 Dana Glens,...,42.7280,-71.1810,47249,Market researcher,1980-11-22,af686a90fb3eff86e0a961d852550619,1365802461,43.044230,-71.136907,0
2,1287799,2020-06-17 16:24:29,4312133045694601139,fraud_Fritsch and Sons,health_fitness,21.84,Julian,Price,M,086 Pierce Cove,...,39.3764,-76.7290,33917,Company secretary,1977-06-12,f70fea505c5a765825aafb72d18c1aa5,1371486269,39.365746,-77.165503,0
3,640815,2019-09-29 19:34:07,30029052116970,fraud_Wiza LLC,misc_pos,8.16,Curtis,Young,M,4319 Watson Shoals Suite 658,...,42.1239,-79.1895,3833,Metallurgist,1970-10-09,61d6b585be9cad4fd36409019b693aaa,1348947247,42.692928,-78.435859,0
4,668880,2019-10-12 14:38:12,3500969075198072,fraud_Predovic Inc,shopping_net,184.55,Kenneth,Sanchez,M,0110 Ashley Forest,...,47.2271,-117.0819,895,Clothing/textile technologist,1999-05-31,e7fc0dc8be81974dacd6fc4c7e3c5362,1350052692,47.861069,-116.983182,0


In [2]:
test = pd.read_csv('../data/preliminary/test.csv')
test.info()
test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             30000 non-null  int64  
 1   trans_date_trans_time  30000 non-null  object 
 2   cc_num                 30000 non-null  int64  
 3   merchant               30000 non-null  object 
 4   category               30000 non-null  object 
 5   amt                    30000 non-null  float64
 6   first                  30000 non-null  object 
 7   last                   30000 non-null  object 
 8   gender                 30000 non-null  object 
 9   street                 30000 non-null  object 
 10  city                   30000 non-null  object 
 11  state                  30000 non-null  object 
 12  zip                    30000 non-null  int64  
 13  lat                    30000 non-null  float64
 14  long                   30000 non-null  float64
 15  ci

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,315706,2020-10-19 12:30:31,3566094707272327,"fraud_Hintz, Bauch and Smith",health_fitness,38.09,Kristina,Stewart,F,47029 Jimmy Tunnel Apt. 106,...,34.3795,-118.5230,34882,Health physicist,1971-04-25,bc12159fcac8af1773fef1d8a1302731,1382185831,34.809923,-118.111582,0
1,151754,2020-08-13 05:57:47,2610529083834453,fraud_Hackett-Lueilwitz,grocery_pos,206.44,Tanya,Dickerson,F,053 Kim Valley Suite 928,...,36.2416,-86.6117,22191,Prison officer,1994-07-27,3b1d9ac4a26f3c56b0cf4cc407250f8d,1376373467,36.644354,-87.480427,0
2,189099,2020-08-26 05:06:42,4873783502705038,fraud_Bahringer-Larson,grocery_net,51.76,Elizabeth,Maxwell,F,194 Goodman Fall Apt. 569,...,48.4786,-122.3345,14871,Public house manager,1974-03-10,6975ab8f7f26df75cc6046b07f7ceeb5,1377493602,48.784813,-123.198143,0
3,249823,2020-09-20 18:44:30,5540636818935089,fraud_Jakubowski Group,food_dining,89.38,Kenneth,Foster,M,329 Michael Extension,...,42.6911,-71.1605,76383,Geoscientist,1985-04-04,ee19c9619769485a654364e035b4c01f,1379702670,42.893355,-70.856259,0
4,487280,2020-12-16 13:22:16,213163860545705,"fraud_Lowe, Dietrich and Erdman",kids_pets,41.72,Mark,Tyler,M,82201 Bradley Radial Suite 703,...,33.1410,-82.5150,741,Claims inspector/assessor,1986-04-28,dac5f5438277e192aabd02e93ee6e1a1,1387200136,33.597864,-83.314567,0


In [3]:
validation = pd.read_csv('../data/preliminary/validation.csv')
validation.info()
validation.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             30000 non-null  int64  
 1   trans_date_trans_time  30000 non-null  object 
 2   cc_num                 30000 non-null  int64  
 3   merchant               30000 non-null  object 
 4   category               30000 non-null  object 
 5   amt                    30000 non-null  float64
 6   first                  30000 non-null  object 
 7   last                   30000 non-null  object 
 8   gender                 30000 non-null  object 
 9   street                 30000 non-null  object 
 10  city                   30000 non-null  object 
 11  state                  30000 non-null  object 
 12  zip                    30000 non-null  int64  
 13  lat                    30000 non-null  float64
 14  long                   30000 non-null  float64
 15  ci

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,1032241,2020-03-03 14:32:35,4906628655840914250,fraud_Feil-Morar,health_fitness,283.13,Charles,Moreno,M,838 Franklin Prairie Apt. 902,...,24.6557,-81.3824,32891,Town planner,1987-02-13,cad774f9be5e482b8a7eb4bb10c3893d,1362321155,25.512218,-82.292661,0
1,921195,2019-12-30 21:32:22,571365235126,fraud_Jakubowski Group,food_dining,6.57,Barbara,Taylor,F,0069 Robin Brooks Apt. 695,...,44.5995,-86.2141,372,"Exhibitions officer, museum/gallery",1995-07-12,e1cf7e14ffbb8ac4888982f79147694a,1356903142,44.034668,-85.607445,0
2,869168,2019-12-19 23:13:48,4755696071492,fraud_Erdman-Ebert,personal_care,12.23,Robert,Evans,M,01892 Patricia Vista Apt. 828,...,32.9643,-96.6012,20328,Site engineer,1985-06-20,2729c48fe5eb11ab671aa3341236cebc,1355958828,32.598802,-96.689835,0
3,956492,2020-01-19 14:48:11,370612217861404,"fraud_Greenholt, O'Hara and Balistreri",shopping_net,6.35,Amanda,Fitzgerald,F,298 Kayla Lights,...,40.1159,-74.8536,21125,"Administrator, charities/voluntary organisations",1985-05-25,36d2a37aab561bb9335e82f1df4f96c7,1358606891,39.889060,-75.542567,0
4,263801,2019-05-11 13:45:14,4378993458389626,"fraud_Schneider, Hayes and Nikolaus",food_dining,2.93,Travis,Hayes,M,1561 Chase Grove,...,43.9742,-71.1503,3807,Surgeon,1999-10-25,32a8bdeca822ad81674e544dbddc3ef3,1336743914,43.643013,-70.210856,0


In [4]:
# split train df, validation df, test df into each of their X and y
y_train = train.pop('is_fraud')
X_train = train

# y_valid = validation_df1.pop('is_fraud')
# X_valid = validation_df1

# y_test = test_df1.pop('is_fraud')
# X_test = test_df1

In [5]:
from geopy.distance import geodesic

# Function to calculate distance between two coordinates
def calculate_distance(row):
    user_coords = (row['lat'], row['long'])
    merchant_coords = (row['merch_lat'], row['merch_long'])
    return geodesic(user_coords, merchant_coords).km

# Apply this to each row in the dataset to create a 'distance' column
X_train['distance'] = X_train.apply(calculate_distance, axis=1)


In [8]:
# Add an index column to preserve original order
X_train['original_order'] = X_train.index

# Calculate the time since the last transaction without affecting original order
def calculate_time_diff(series):
    return series.diff().fillna(0)  # Fill NaN with 0 for the first transaction

X_train['time_since_last_trans'] = (
    X_train.groupby('cc_num')['unix_time']
    .transform(calculate_time_diff)
)

# Vectorized Distance Calculation
def calculate_previous_distance(group):
    # Get coordinates of previous transactions
    user_lat = group['lat'].shift(1)
    user_long = group['long'].shift(1)
    merchant_lat = group['merch_lat'].shift(1)
    merchant_long = group['merch_long'].shift(1)

    # Calculate distance using geodesic for the previous transaction
    distances = [
        geodesic((user_lat.iloc[i], user_long.iloc[i]), (merchant_lat.iloc[i], merchant_long.iloc[i])).km
        if i > 0 else 0.0  # For the first transaction
        for i in range(len(group))
    ]
    
    return pd.Series(distances)

# Apply the distance calculation function to each group
X_train['dist_from_last_trans'] = (
    X_train.groupby('cc_num')
    .apply(calculate_previous_distance)
).reset_index(drop=True)

# Calculate speed based on the previous distance
X_train['speed'] = np.where(
    X_train['time_since_last_trans'] > 0,
    X_train['dist_from_last_trans'] / X_train['time_since_last_trans'],
    0  # Speed is 0 if time difference is 0
)

# Sort the DataFrame back to its original order using the 'original_order' column
X_train.sort_values(by='original_order', inplace=True)

# Drop the 'original_order' column after sorting
X_train.drop(columns=['original_order'], inplace=True)

# Now X_train has 'time_since_last_trans' and 'previous_distance' with the original order maintained.


In [9]:
pd.set_option('display.max_columns', None)
X_train[['cc_num', 'unix_time', 'lat', 'long', 'merch_lat', 'merch_long', 'distance', 'dist_from_last_trans', 'time_since_last_trans', 'speed']][X_train['cc_num'] == 60416207185]


,cc_num,unix_time,lat,long,merch_lat,merch_long,distance,dist_from_last_trans,time_since_last_trans,speed
106,60416207185,1327251016,43.0048,-108.8964,43.089370,-108.008073,72.986571,9.568144,0.0,0.000000
943,60416207185,1365918316,43.0048,-108.8964,43.092772,-108.880445,9.859210,56.725157,38667300.0,0.000001
2038,60416207185,1357598502,43.0048,-108.8964,43.478092,-108.124252,81.840267,97.349127,-8319814.0,0.000000
2644,60416207185,1371274978,43.0048,-108.8964,42.417253,-109.758052,96.138118,89.028912,13676476.0,0.000007
2911,60416207185,1329948333,43.0048,-108.8964,42.301079,-109.312925,85.308670,104.901420,-41326645.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
237936,60416207185,1343069959,43.0048,-108.8964,43.422146,-109.283609,56.033465,136.451667,-7668328.0,0.000000
238692,60416207185,1346520114,43.0048,-108.8964,43.255903,-109.744092,74.402517,37.244523,3450155.0,0.000011
239018,60416207185,1369351743,43.0048,-108.8964,43.236356,-109.765999,75.299152,122.261012,22831629.0,0.000005
239334,60416207185,1332403770,43.0048,-108.8964,43.327897,-109.635006,69.971790,102.329513,-36947973.0,0.000000


In [10]:
X_train

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,distance,time_since_last_trans,dist_from_last_trans,speed
0,1072349,2020-03-21 19:05:27,2242542703101233,fraud_Bahringer Group,health_fitness,1.40,Samuel,Jenkins,M,43235 Mckenzie Views Apt. 837,Westport,KY,40077,38.4921,-85.4524,564,Pensions consultant,1996-04-10,5a4c5bc47fd9ca60a9cdbe38423654b4,1363892727,37.965322,-85.661501,61.272074,0.0,0.000000,0.000000
1,1123019,2020-04-12 21:34:21,4469777115158234880,"fraud_Douglas, Schneider and Turner",shopping_pos,9.73,Gregory,Graham,M,4005 Dana Glens,Methuen,MA,1844,42.7280,-71.1810,47249,Market researcher,1980-11-22,af686a90fb3eff86e0a961d852550619,1365802461,43.044230,-71.136907,35.314334,0.0,72.986571,0.000000
2,1287799,2020-06-17 16:24:29,4312133045694601139,fraud_Fritsch and Sons,health_fitness,21.84,Julian,Price,M,086 Pierce Cove,Pikesville,MD,21208,39.3764,-76.7290,33917,Company secretary,1977-06-12,f70fea505c5a765825aafb72d18c1aa5,1371486269,39.365746,-77.165503,37.632941,0.0,9.859210,0.000000
3,640815,2019-09-29 19:34:07,30029052116970,fraud_Wiza LLC,misc_pos,8.16,Curtis,Young,M,4319 Watson Shoals Suite 658,Falconer,NY,14733,42.1239,-79.1895,3833,Metallurgist,1970-10-09,61d6b585be9cad4fd36409019b693aaa,1348947247,42.692928,-78.435859,88.566234,0.0,81.840267,0.000000
4,668880,2019-10-12 14:38:12,3500969075198072,fraud_Predovic Inc,shopping_net,184.55,Kenneth,Sanchez,M,0110 Ashley Forest,Tekoa,WA,99033,47.2271,-117.0819,895,Clothing/textile technologist,1999-05-31,e7fc0dc8be81974dacd6fc4c7e3c5362,1350052692,47.861069,-116.983182,70.876256,0.0,96.138118,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,589692,2019-09-07 16:09:20,4079773899158,fraud_Waelchi Inc,kids_pets,12.88,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,ID,83643,44.6255,-116.4493,129,Cartographer,1965-12-15,5dbc1fa717e840c81823e095d955bbbf,1347034160,44.798352,-116.632527,24.078302,-11452207.0,85.533768,0.000000
239996,314690,2019-06-01 06:52:28,180064817343047,"fraud_Schuppe, Nolan and Hoeger",shopping_pos,1.63,Patricia,Gonzalez,F,8973 Mark Drive,Stephensport,KY,40170,37.9048,-86.5240,344,Chartered public finance accountant,1967-03-30,8eaf8a4778b3e680c65d518543926b06,1338533548,38.157359,-85.857702,64.867855,-1770545.0,114.523139,0.000000
239997,1194213,2020-05-13 20:14:00,30408301059761,fraud_Hilpert-Conroy,kids_pets,40.69,Brandon,Pittman,M,2881 Frey Village,Cross,SC,29436,33.3364,-80.1859,4471,Occupational hygienist,1973-10-09,d8b06bc69af2831cbd158ab5ca7bd200,1368476040,33.821302,-81.055067,96.972398,26106912.0,80.385202,0.000003
239998,1073052,2020-03-22 02:48:25,38057513087029,fraud_Luettgen PLC,gas_transport,47.25,Marissa,Clark,F,2677 Byrd Village Suite 035,San Jose,CA,95148,37.3304,-121.7913,973849,Geoscientist,1997-01-18,3100d31fbc201be7bdd07faa0ee1d37b,1363920505,38.272782,-122.041386,106.891090,-792749.0,78.713450,0.000000
